# NLP with RoBERTa.

Yeah, I am somewhat of an NLP engineer myself

In [1]:
import torch
import json
import torch.nn.functional as F
import pandas as pd
from fairseq.data.data_utils import collate_tokens
from tqdm.auto import tqdm
import numpy as np

In [2]:
omdb = json.load(open("../../../../data/parsed/omdb.json", "r") )
tmdb = json.load(open("../../../../data/parsed/tmdb.json", "r") )

In [3]:
batch_size = 4
cuda = torch.device('cuda')

In [4]:
plots = []
for i in tmdb.keys():
    omdb_plot = omdb[i]['omdb'].get('Plot', '')
    tmdb_plot = tmdb[i]['tmdb'].get('overview', '')
    plot = tmdb_plot + ' ' + omdb_plot
    plots.append((i, plot, len(plot)))
    
plots = list(sorted(plots, key=lambda x: x[2]))
plots = list(filter(lambda x: x[2] > 4, plots))

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

ids = [i[0] for i in plots]
plots = [i[1] for i in plots]
plots = list(chunks(plots, batch_size))
ids = list(chunks(ids, batch_size))

In [5]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.base').to(cuda)
roberta.eval()
print()

Using cache found in /home/dev/.cache/torch/hub/pytorch_fairseq_master


loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.base.tar.gz from cache at /home/dev/.cache/torch/pytorch_fairseq/37d2bc14cf6332d61ed5abeb579948e6054e46cc724c7d23426382d11a31b2d6.ae5852b4abc6bf762e0b6b30f19e741aa05562471e9eb8f4a6ae261f04f9b350
| dictionary: 50264 types



In [6]:
fs = {}

def extract_features(batch, ids):
    batch = collate_tokens([roberta.encode(sent) for sent in batch], pad_idx=1).to(cuda)
    batch = batch[:, :512]
    features = roberta.extract_features(batch)
    pooled_features = F.avg_pool2d(features, (features.size(1), 1)).squeeze()
    for i in range(pooled_features.size(0)):
        fs[ids[i]] = pooled_features[i].detach().cpu().numpy()

In [7]:
for batch, ids in tqdm(zip(plots[::-1], ids[::-1]), total=len(plots)):
    extract_features(batch, ids)

In [15]:
transformed = pd.DataFrame(fs).T
transformed.index = transformed.index.astype(int)
transformed = transformed.sort_index()

In [16]:
transformed.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
1,-0.005599,0.138494,0.047051,-0.099981,0.208267,0.163597,-0.050247,0.035369,0.021860,-0.001333,...,-0.053477,0.014401,-0.035731,-0.068612,0.146932,0.106177,-0.128289,-0.231606,0.047912,-0.046285
2,-0.028936,0.053734,0.066000,-0.130739,0.197591,0.014505,-0.001784,0.091164,0.036338,-0.002871,...,-0.058495,0.049999,-0.049668,-0.037801,0.088053,0.142559,-0.166629,-0.081439,0.034168,-0.023142
3,0.023951,0.082014,0.041002,-0.058334,0.188524,0.099200,0.009292,0.044268,0.051445,0.032975,...,-0.031117,-0.017112,-0.016568,-0.009261,0.070678,0.122078,-0.029504,-0.045054,0.114256,0.064617
4,0.028417,0.169414,0.063841,-0.036933,0.114328,0.082039,0.017422,0.084967,-0.001609,0.048082,...,-0.081082,-0.044695,0.164680,0.029210,0.015597,0.080508,0.006273,-0.155380,0.039771,0.049289
5,0.011459,0.131149,0.039703,-0.037407,0.289072,0.121404,-0.046844,-0.013482,-0.103010,0.039538,...,-0.075606,0.007551,0.031218,-0.000565,0.113364,0.092764,0.033090,-0.285467,0.050361,0.061391


In [18]:
transformed.to_csv('../../../../data/engineering/roberta.csv', index=True, index_label='idx')